# Segmenting and Clustering Neighborhoods in Toronto 


## Part 3 - Clustering Analysis.

In this part we are going to be following the example of the New York analysis and clustering.

First let's import all the necessary libraries we need to use.

In [79]:
import numpy as np # library to handle data in a vectorized manner
from itertools import chain
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [80]:
import geocoder

In [81]:
toronto = pd.read_csv('data1.csv')
toronto = toronto.drop('Unnamed: 0', axis=1)

# As suggested we will only keep the Boroughs that contain the word "Toronto"
toronto = toronto[toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)

# The final dataframe size
toronto.shape
toronto = toronto.drop('Latitude', axis=1)
toronto = toronto.drop('Longitude', axis=1)
toronto

,Postcode,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,"The Danforth West, Riverdale"
2,M4L,East Toronto,"The Beaches West, India Bazaar"
3,M4M,East Toronto,Studio District
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North
6,M4R,Central Toronto,North Toronto West
7,M4S,Central Toronto,Davisville
8,M4T,Central Toronto,"Moore Park, Summerhill East"
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."


## Pre - Processing

**Because of previous processing, some rows have more than one neighborhoods, we are going to seperate each neighborhood, keeping it's Borough and Postcode the same**

Let's see how many boroughs and neighborhoods we have now:

In [82]:
def chainer(s):
    return list(chain.from_iterable(s.str.split(',')))

lens = toronto['Neighborhood'].str.split(',').map(len)

toronto = pd.DataFrame({'Postcode': np.repeat(toronto['Postcode'], lens),
                        'Borough': np.repeat(toronto['Borough'], lens),
                        'Neighborhood': chainer(toronto['Neighborhood']),
    
})

print(toronto)

             Borough                                       Neighborhood  \
0       East Toronto                                        The Beaches   
1       East Toronto                                  The Danforth West   
1       East Toronto                                          Riverdale   
2       East Toronto                                   The Beaches West   
2       East Toronto                                       India Bazaar   
3       East Toronto                                    Studio District   
4    Central Toronto                                      Lawrence Park   
5    Central Toronto                                   Davisville North   
6    Central Toronto                                 North Toronto West   
7    Central Toronto                                         Davisville   
8    Central Toronto                                         Moore Park   
8    Central Toronto                                    Summerhill East   
9    Central Toronto     

In [83]:
toronto = toronto[['Postcode', 'Borough', 'Neighborhood']]
toronto

,Postcode,Borough,Neighborhood
0,M4E,East Toronto,The Beaches
1,M4K,East Toronto,The Danforth West
1,M4K,East Toronto,Riverdale
2,M4L,East Toronto,The Beaches West
2,M4L,East Toronto,India Bazaar
3,M4M,East Toronto,Studio District
4,M4N,Central Toronto,Lawrence Park
5,M4P,Central Toronto,Davisville North
6,M4R,Central Toronto,North Toronto West
7,M4S,Central Toronto,Davisville


In [84]:
def get_latlng(postal_code):
    
    # initialize your variable to None
    lat_lng_coords = None
    
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format
                            (postal_code))
        lat_lng_coords = g.latlng
        
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [85]:
postcodes = toronto['Postcode']    
coords = [ get_latlng(postcode) for postcode in postcodes.tolist() ]

In [86]:
df_coords = pd.DataFrame(coords, columns=['latitude', 'longitude'])
toronto['Latitude'] = df_coords['latitude']
toronto['Longitude'] = df_coords['longitude']


In [87]:
toronto.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676845,-79.295225
1,M4K,East Toronto,The Danforth West,43.683262,-79.355120
1,M4K,East Toronto,Riverdale,43.683262,-79.355120
2,M4L,East Toronto,The Beaches West,43.683262,-79.355120
2,M4L,East Toronto,India Bazaar,43.683262,-79.355120


In [88]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto['Borough'].unique()),
        toronto.shape[0]
    )
)

The dataframe has 4 boroughs and 74 neighborhoods.


### Now we can visualize Toronto as well as the neighorhoods surrounding it. 
But first we will need to get the lon,lat values of Toronto using the geocoder library.

In [89]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


**Create a map of Toronto with neighborhoods superimposed on top**

In [90]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Next we are going to utilize the Foursquare API to explore the neighborhoods and segment them.

_The following cell is hidden as it containts the Foursquare credentials

In [91]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: SCIFWVX3WWS3XB4A02IZ3VKWNFMAX1PJT3D1RXHDXSOAZEAC
CLIENT_SECRET:4XGA2VX2HUIEMNOBBF0ZRUTKM2WE12NARHW4FR2QNRF13SMM


Let's get all the nearby venues for each neighborhood

In [92]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [93]:
toronto_venues = getNearbyVenues(names=toronto['Neighborhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

The Beaches
The Danforth West
 Riverdale
The Beaches West
 India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park
 Summerhill East
Deer Park
 Forest Hill SE
 Rathnelly
 South Hill
 Summerhill West
Rosedale
Cabbagetown
 St. James Town
Church and Wellesley
Harbourfront
 Regent Park
Ryerson
 Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide
 King
 Richmond
Harbourfront East
 Toronto Islands
 Union Station
Design Exchange
 Toronto Dominion Centre
Commerce Court
 Victoria Hotel
Roselawn
Forest Hill North
 Forest Hill West
The Annex
 North Midtown
 Yorkville
Harbord
 University of Toronto
Chinatown
 Grange Park
 Kensington Market
CN Tower
 Bathurst Quay
 Island airport
 Harbourfront West
 King and Spadina
 Railway Lands
 South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place
 Underground city
Christie
Dovercourt Village
 Dufferin
Little Portugal
 Trinity
Brockton
 Exhibition Place
 Parkdale Village
High Park
 The 

#### This is the resulting Dataframe and it's size

In [94]:
print(toronto_venues.shape)
toronto_venues.head()

(3559, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676845,-79.295225,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676845,-79.295225,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676845,-79.295225,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Danforth West,43.683262,-79.355120,Dairy Queen,43.684223,-79.357062,Fast Food Restaurant
4,The Danforth West,43.683262,-79.355120,Dollarama,43.686300,-79.355893,Discount Store


In [95]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bathurst Quay,100,100,100,100,100,100
Dufferin,7,7,7,7,7,7
Exhibition Place,7,7,7,7,7,7
Forest Hill SE,26,26,26,26,26,26
Forest Hill West,100,100,100,100,100,100
Garden District,7,7,7,7,7,7
Grange Park,62,62,62,62,62,62
Harbourfront West,100,100,100,100,100,100
India Bazaar,5,5,5,5,5,5


## Analyze Each Neighborhood

In [96]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Danforth West,0,0,0,0,0,0,0,0,0,0,

In [97]:
toronto_onehot.shape

(3559, 185)

Groupping rows by neighborhood and by taking the mean of the frequency of the occurence of each category

In [98]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Camera Store,Cheese Shop,Chinese Restaurant,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,General Travel,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health Food Store,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,Bathurst Quay,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.010000,0.000000,0.020000,0.0,0.010000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.020000,0.00,0.060000,0.000000,0.110000,0.00,0.00,0.000000,0.01,0.000000,0.000000,0.04,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.01,0.010000,0.010000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.010000,0.00,0.010000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.010000,0.010000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.03,0.010000,0.01,0.000000,0.00,0.010000,0.000000,0.010000,0.00,0.000000,0.00,0.01,0.00,0.00,0.000000,0.010000,0.00,0.000000,0.000000,0.020000,0.000000,0.020000,0.01,0.00,0.010000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.01,0.010000,0.000000,0.000000,0.00,0.000000,0.00,0.020000,0.00,0.02,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.00,0.000000,0.000000,0.01,0.030000,0.0,0.010000,0.010000,0.000000,0.010000,0.0,0.00

Let's confirm the size

In [99]:
toronto_grouped.shape

(74, 185)

## Top 10 most common venues for each neighborhood

In [100]:
num_top_venues = 10

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Bathurst Quay----
                       venue  freq
0                Coffee Shop  0.11
1             Clothing Store  0.06
2             Cosmetics Shop  0.04
3  Middle Eastern Restaurant  0.03
4           Sushi Restaurant  0.03
5                   Tea Room  0.03
6                Pizza Place  0.02
7        Sporting Goods Shop  0.02
8         Chinese Restaurant  0.02
9                       Café  0.02


---- Dufferin----
                             venue  freq
0                  Harbor / Marina  0.29
1                             Park  0.29
2              American Restaurant  0.14
3               Athletics & Sports  0.14
4                      Music Venue  0.14
5                      Yoga Studio  0.00
6                     Noodle House  0.00
7       Modern European Restaurant  0.00
8  Molecular Gastronomy Restaurant  0.00
9              Monument / Landmark  0.00


---- Exhibition Place----
                             venue  freq
0                  Harbor / Marina  0.29
1          

In [101]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [102]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bathurst Quay,Coffee Shop,Clothing Store,Cosmetics Shop,Middle Eastern Restaurant,Sushi Restaurant,Tea Room,Restaurant,Plaza,Sporting Goods Shop,Bakery
1,Dufferin,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
2,Exhibition Place,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
3,Forest Hill SE,Dessert Shop,Italian Restaurant,Pizza Place,Coffee Shop,Café,Sandwich Place,Fast Food Restaurant,Park,Diner,Restaurant
4,Forest Hill West,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Middle Eastern Restaurant,Ramen Restaurant,Tea Room,Japanese Restaurant,Lingerie Store,Hotel


## Clustering the Neighborhoods

We are going to run _k-means_ to cluster the neighborhoods into 6 clusters

In [103]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 2, 1, 1, 3, 1, 1, 1, 1], dtype=int32)

Let's modify our dataframe to include the cluster label as well as the 10 most common venues

In [104]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676845,-79.295225,1,Health Food Store,Pub,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,M4K,East Toronto,The Danforth West,43.683262,-79.355120,1,Discount Store,Fast Food Restaurant,Park,Grocery Store,Bus Line,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
1,M4K,East Toronto,Riverdale,43.683262,-79.355120,1,Discount Store,Fast Food Restaurant,Park,Grocery Store,Bus Line,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
2,M4L,East Toronto,The Beaches West,43.683262,-79.355120,1,Discount Store,Fast Food Restaurant,Park,Grocery Store,Bus Line,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
2,M4L,East Toronto,India Bazaar,43.683262,-79.355120,1,Discount Store,Fast Food Restaurant,Park,Grocery Store,Bus Line,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


And finally let's visualize our results in a Map

In [105]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [119]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + toronto_merged.columns[2] +list(range(5, toronto_merged.shape[1]))]]

TypeError: can only concatenate list (not "str") to list

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Health Food Store,Pub,Dog Run,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
1,East Toronto,1,Discount Store,Fast Food Restaurant,Park,Grocery Store,Bus Line,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
1,East Toronto,1,Discount Store,Fast Food Restaurant,Park,Grocery Store,Bus Line,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
2,East Toronto,1,Discount Store,Fast Food Restaurant,Park,Grocery Store,Bus Line,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
2,East Toronto,1,Discount Store,Fast Food Restaurant,Park,Grocery Store,Bus Line,Eastern European Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
3,East Toronto,1,Pizza Place,Liquor Store,Pub,Fast Food Restaurant,Fish & Chips Shop,Sandwich Place,Steakhouse,Sushi Restaurant,Movie Theater,Burrito Place
4,Central Toronto,1,Pizza Place,Liquor Store,Pub,Fast Food Restaurant,Fish & Chips Shop,Sandwich Place,Steakhouse,Sushi Restaurant,Movie Theater,Burrito Place
5,Central Toronto,1,Italian Restaurant,Diner,Café,Bakery,Coffee Shop,Brewery,Sushi Restaurant,Bar,Record Shop,Gastropub
6,Central Toronto,1,Swim School,Bus Line,Lake,Women's Store,Flea Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
7,Central Toronto,1,Clothing Store,Park,Food & Drink Shop,Gym,Breakfast Spot,Hotel,Dance Studio,Fish & Chips Shop,Cycle Studio,Fast Food Restaurant


In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,2,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
31,West Toronto,2,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
32,West Toronto,2,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
32,West Toronto,2,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
33,West Toronto,2,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
33,West Toronto,2,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
33,West Toronto,2,Harbor / Marina,Park,Music Venue,American Restaurant,Athletics & Sports,Women's Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [110]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Downtown Toronto,3,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Women's Store,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
13,Downtown Toronto,3,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Women's Store,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
13,Downtown Toronto,3,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Women's Store,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
14,Downtown Toronto,3,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Women's Store,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
14,Downtown Toronto,3,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Women's Store,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
15,Downtown Toronto,3,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Women's Store,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
16,Downtown Toronto,3,Light Rail Station,Coffee Shop,Liquor Store,Supermarket,Athletics & Sports,Women's Store,Electronics Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


We can tell that the clusters 